In [ ]:
import fastf1
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.express as px
import numpy as np

session = fastf1.get_session(2024, 'Abu Dhabi', 'R')
session.load()
laps = session.laps.pick_quicklaps()


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
_api        WARNING 	Driver  2: Car data is incomplete!
_api        WARNING 	Driver  3: Car data is incomplete!
_api        WARNING 	Driver  5: Car data is incomplete!
_api        WARNING 	Driver  6: Car data is incomplete!
_api        WARNING 	Driver  7: Car data is incomplete!
_api        WARNING 	Driver  8:

In [9]:
tires = ['SOFT', 'MEDIUM', 'HARD']
prediction_data = []

for compound in tires:
    comp_laps = laps[laps['Compound'] == compound].dropna(subset = ['LapTime', 'TyreLife'])
    if len(comp_laps) < 10: continue

    X, y = torch.tensor(comp_laps['TyreLife'].values, dtype=torch.float32).reshape(-1, 1), torch.tensor(comp_laps['LapTime'].dt.total_seconds().values, dtype = torch.float32).reshape(-1, 1)

    model = nn.Linear(in_features=1, out_features=1)
    optimizer = optim.SGD(model.parameters(), lr=0.0005)
    criterion = nn.MSELoss()
    epochs = 2000
    for epoch in range(epochs):
        predictions = model(X)
        loss = criterion(predictions, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    virtual_laps = torch.arange(0, 41, dtype=torch.float32).reshape(-1, 1)
    with torch.no_grad():
        predicted_pace = model(virtual_laps).numpy().flatten()
    
    for age, pace in zip(virtual_laps.flatten().numpy(), predicted_pace):
        prediction_data.append({
            'Compound': compound,
            'Tire Age': age,
            'Predicted Pace (s)': pace
        })
    print(f"{compound:<10} | {model.weight.item():+.3f}               | {model.bias.item():.2f}")

SOFT       | +3.603               | 31.60
MEDIUM     | +3.804               | 30.58
HARD       | +2.337               | 35.70


In [12]:
df_pred = pd.DataFrame(prediction_data)
fig = px.line(df_pred,
             x='Tire Age',
             y='Predicted Pace (s)',
            title='Tire Degradation Curves Prediction',
            color='Compound',
            template='plotly_dark',
            color_discrete_map={'SOFT': 'red', 'MEDIUM': 'yellow', 'HARD': 'white'})

#fig.update_layout(yaxis_title = "lap time (lower is faster)")

fig.show()